In [1]:
import dicom2nifti
from rt_utils import RTStructBuilder

In [2]:
import nibabel as nib
import glob
import numpy as np
import re
import pandas as pd
import os
import math
import pydicom as pdcm

In [3]:
import yaml
with open('config_Dicom2Nii.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    
    itvtot = config['itv_tot_labels']
    itvtum = config['itv_tumor_labels']
    itvnod = config['itv_ln_labels']
    
    gtvtot = config['gtv_tot_labels']
    gtvtum = config['gtv_tumor_labels']
    gtvnod = config['gtv_ln_labels']

    root_nii = config['save_path']
    
    #source_csv_path = config['source_csv']
    source_csv_path = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/Dicom2Nii_LastRound.csv"
    
    
    save_path = config['save_path']

df = pd.read_csv(source_csv_path)
target_labels = itvtot + itvtum+itvnod+gtvtot+gtvtum+gtvnod

In [4]:
df['pxID'] = df['pxID'].astype(int)
id_column = df.iloc[:, 0]
df

,pxID,CT,ITVTot,ITVTumor,ITVLN,GTVtot,GTVTumor,GTVLN,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%
0,13194,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
1,23840,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...
2,102687,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
3,180869,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...
4,248219,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdat

In [5]:
df["CT"][0]

'//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0013194\\\\5804768.1/Thorax 2mm  standaard  B40f/'

In [6]:
def LookForPlanCT_GTV_ITV_InNii(path_Nii):
    missingCT_bool,missingITV_bool,missingGTV_bool = (True,True,True)
    for f in os.listdir(path_Nii):
        if "ave" in f.lower() or "mip" in f.lower() or "thorax" in f.lower():
            missingCT_bool= False
        if "gtv" in f.lower():
            missingGTV_bool = False
        if "itv" in f.lower() or "igtv" in f.lower():
            missingITV_bool = False    
    return missingCT_bool,missingITV_bool,missingGTV_bool

In [12]:
count=0
pattern = re.compile(r'(?![2]),(?![0-9])')
for patientID in id_column:
    #COLLECT AND FORMAT THE DATA 
    PlanCT_path = df[id_column==patientID]['CT'].iloc[0]
    PlanCT_path_list = pattern.split(PlanCT_path)
    PlanCT_path_clean = [element.strip() for element in PlanCT_path_list]
    PlanCT_path_unique = list(set(PlanCT_path_clean))
    
    listCTs_path = df[df['pxID'] == patientID].iloc[:,8:20].values.tolist()
    listCTs_path_split = [item for sublist in listCTs_path for item in sublist]

    listRTs_path = df[df['pxID'] == patientID].iloc[:, 2:8].values.tolist()
    listRTs_path_split = [item for sublist in listRTs_path for item in sublist]
    listRTs_path_unique = np.unique(listRTs_path_split)

    #CREATE PATH
    if not os.path.exists(save_path+str(patientID)):
        os.makedirs(save_path+str(patientID)+'/')
        print("NEW Px:",patientID,"Count",count)

    #CHECK IF PX has been completed before
    missPlanCTNii,missITVNii,missGTVTNii = LookForPlanCT_GTV_ITV_InNii(save_path+str(patientID))
    if False and not(missPlanCTNii):
        print("Already Complete Px:",patientID,"Count",count)
    else:
        print("Not complete Folder Px:",patientID,"Count",count)
        save_px_path = os.path.join(save_path,str(patientID))
        
        #Convert Plan CT
        print("Plan CT")
        for singPlanCT_path in PlanCT_path_unique:
            print(singPlanCT_path.split)
            dicom2nifti.convert_directory(singPlanCT_path, save_px_path, compression=True, reorient=False)
            planCT_Nii_path = os.listdir(save_px_path)

        # RTSTruct
        try:
            print("RTStruct:")
            ct_nifti = nib.load(os.path.join(save_px_path,planCT_Nii_path[0]))
            for singPlanCT_path in PlanCT_path_unique:
                for singleRT_path in listRTs_path_unique:
                    if not(str(singleRT_path)=="nan"):
                        print(singleRT_path)
                        try:
                            rtstructUTILS = RTStructBuilder.create_from(dicom_series_path=singPlanCT_path,rt_struct_path=singleRT_path.replace("\\\\", "/"))
                            all_names = rtstructUTILS.get_roi_names()
                        except Exception as e:
                                print("ERROR at RTUtils",e)
                                pass
                    for curr_mask in target_labels: 
                        for curr_name in all_names: 
                            if curr_mask == curr_name:
                                try:
                                    mask_3d = rtstructUTILS.get_roi_mask_by_name(curr_mask)
                                    saved_mask = curr_mask
                                    mask_nifti = nib.Nifti1Image(mask_3d, affine=ct_nifti.affine, header=ct_nifti.header)
                                    nib.save(mask_nifti, save_px_path+'/'+saved_mask+'.nii.gz')
                                except Exception as e:
                                    print("ERROR at mask Conv",e)
                                    pass
        except Exception as e:
                print("ERROR at RT Struct and PlanNii(NoCT Created):",e)
                pass

        #CONVERT BP CTS
        print("BP CTs")
        for singleCT_path in listCTs_path_split:
            if not(str(singleCT_path)=="nan"):
                listofCTs = singleCT_path.split(", ")
                for currSingleCT_path in listofCTs:
                    print(currSingleCT_path)
                    dicom2nifti.convert_directory(currSingleCT_path, save_px_path, compression=True, reorient=False)
                    
    count+=1 

Not complete Folder Px: 13194 Count 0
Plan CT
<built-in method split of str object at 0x000001FD4CC35CB0>
RTStruct:
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0013194\\5804768.1/PinnPlan = A1Long (Wed Sep 09 142123 2015) RTSTRUCT/2.16.840.1.113669.2.931128.289629209.20150909144342.40809.DCM
ERROR at mask Conv OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2431: error: (-215:Assertion failed) p.checkVector(2, CV_32S) >= 0 in function 'cv::fillPoly'

ERROR at mask Conv 'Dataset' object has no attribute 'ContourSequence'
ERROR at mask Conv 'Dataset' object has no attribute 'ContourSequence'
ERROR at mask Conv OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2431: error: (-215:Assertion failed) p.checkVector(2, CV_32S) >= 0 in function 'cv::fillPoly'

ERROR at mask Conv 'Dataset' object has no attribute 'ContourSequence'
ERROR at mask Conv OpenCV(4.8.1) D:\a\opencv


//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0180869\\5861864.1/4D thorax 2.0 2.0 I30f 3 0 - 100  TRIGGER_DELAY 80%/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0180869\\5861864.1/4D thorax 2.0 2.0 I30f 3 0 - 100  TRIGGER_DELAY 90%/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0180869\\5861864.1/top  4D thorax 2.0  2.0  I30f  3  95% In/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0180869\\5861864.1/4D thorax 2.0 2.0 I30f 3 0 - 100  TRIGGER_DELAY 100%/

Not complete Folder Px: 248219 Count 4
Plan CT
<built-in method split of str object at 0x000001FD4CC35FB0>


Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
R

RTStruct:
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0248219\\X20 THORAX/PinnPlan = A1LongParIMRT (Fri Nov 21 111812 2014) RTSTRUCT/2.16.840.1.113669.2.931128.289629209.20141121113308.726200.DCM
BP CTs
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0248219\\0/RespLow  2.0  B30s  100% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0248219\\0/RespLow  2.0  B30s  60% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0248219\\0/RespLow  2.0  B30s  30% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0248219\\0/RespLow  2.0  B30s  0% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0248219\\0/RespLow  2.0  B30s  5% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG


Not complete Folder Px: 367466 Count 9
Plan CT
<built-in method split of str object at 0x000001FD4CC361F0>
RTStruct:
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0367466\\0/PinnPlan = A1Long (Fri Aug 14 120835 2015) RTSTRUCT/2.16.840.1.113669.2.931128.289629209.20150814123012.987655.DCM
BP CTs
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0367466\\0/RespLow  2.0  B30s  100% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0367466\\0/RespLow  2.0  B30s  60% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0367466\\0/RespLow  2.0  B30s  30% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0367466\\0/RespLow  2.0  B30s  0% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0367466\\0/RespLow  2.0  B

...

BP CTs
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  100% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  60% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  30% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  0% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  5% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  0% In/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9193538\\X3 THORAX/RespLow  2.0  B30s  40% In/

//z

Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
R

RTStruct:
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9243882\\5579146.1/PinnPlan = A1longen (Fri Feb 13 102709 2015) RTSTRUCT/2.16.840.1.113669.2.931128.289629209.20150213104416.697537.DCM
BP CTs
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9243882\\0/RespLow  2.0  B30s  100% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9243882\\0/RespLow  2.0  B30s  60% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9243882\\0/RespLow  2.0  B30s  30% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9243882\\0/RespLow  2.0  B30s  0% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9243882\\0/RespLow  2.0  B30s  5% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstr


//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  60% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  30% Ex/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  0% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  5% Ex/
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  0% In/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  40% In/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9549714\\X10 THORAX/RespLow  2.0  B30s  90% In/

Not

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9829221\\X1 THORAX/RespLow  2.0  B30s  0% In/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9829221\\X1 THORAX/RespLow  2.0  B30s  40% In/

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9829221\\X1 THORAX/RespLow  2.0  B30s  90% In/

Not complete Folder Px: 9854449 Count 115
Plan CT
<built-in method split of str object at 0x000001FD4CCEDD70>
RTStruct:
//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/9854449\\X2 THORAX/PinnPlan = A1LongLi (Wed Feb 18 141837 2015) RTSTRUCT/2.16.840.1.113669.2.931128.289629209.20150218143553.811349.DCM
ERROR at mask Conv 'Dataset' object has no attribute 'ContourSequence'
ERROR at mask Conv 'Dataset' object has no attribute 'ContourSequence'
ERROR at mask Conv 'Dataset' object has no attribute 'ContourSequence'
BP CTs
//zkh

//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/0013194\\5804768.1/Thorax 2mm  standaard  B40f/
None


SyntaxError: 'break' outside loop (4134452972.py, line 5)

In [11]:
planCT_Nii_path

['10_resplow__20__b30s__100_ex.nii.gz',
 '11_resplow__20__b30s__60_ex.nii.gz',
 '12_resplow__20__b30s__30_ex.nii.gz',
 '13_resplow__20__b30s__5_ex.nii.gz',
 '14_resplow__20__b30s__0_ex.nii.gz',
 '3_thorax_2mm__standaard__b40f.nii.gz',
 '7_resplow__20__b30s__0_in.nii.gz',
 '8_resplow__20__b30s__40_in.nii.gz',
 '9_resplow__20__b30s__90_in.nii.gz']